In [1]:
import pynbody
import os
from matplotlib import pyplot as plt
import numpy as np
import traceback

from akaxia import Sims  
from pynbody.plot.sph import image

import pynbody.plot.sph as sph

# Base directory to store images
base_image_dir = '../Figures/halo_check_images/'

# Ensure the base directory exists
if not os.path.exists(base_image_dir):
    os.makedirs(base_image_dir)

# Parameters for focused testing - specify the simulation and halo ID you want to test
test_sim_name = 'r431'  # Example: 'r431'
test_halo_id = 1  # Example: 1, adjust based on your needs

good_halos = []

# Set a mass threshold
mass_threshold = 1e4

sims = 'storm'

data_path = Sims[sims]['path']
print(data_path)
sim = pynbody.load(data_path)
sim.physical_units()
h = sim.halos(write_fpos=False)
print(len(h))

/data/akaxia/storm/storm.cosmo25cmbsi2s50v35.4096/storm.cosmo25cmbsi2s50v35.4096.004096


pynbody.halo : An error occurred while reading substructure file. Falling back to using the halo info.


8880


In [2]:
import logging
import tqdm
import time

# Set up logging
logging.basicConfig(filename='halo_processing_SIDM.log', level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')



for halo_id in tqdm.tqdm(range(1, len(h)+1), desc="Processing Halos"):
    try:
        start_time = time.time()  # Start timer for the entire loop iteration
    
        # Load the halo
        load_start = time.time()
        halo = h.load_copy(halo_id)
        load_duration = time.time() - load_start
        n = len(halo)
        if  n < 1e4:
            logging.info(f'particle count under threshold for halo {halo_id}: {n}')
            continue
        # Quickly calculate total mass to check against the threshold
        mass_calc_start = time.time()
        total_mass = halo.dm['mass'].in_units('Msol').sum()
        mass_calc_duration = time.time() - mass_calc_start
    
        if total_mass < mass_threshold:
            logging.info(f'Mass under threshold for halo {halo_id}: {total_mass}')
            continue
    
        # Convert all particle masses to solar masses and filter out heavy particles
        filter_start = time.time()
        particle_masses_in_solar = halo.dm['mass'].in_units('Msol')
        heavy_particles_mask = particle_masses_in_solar > 1e4
        heavy_particle_mass = particle_masses_in_solar[heavy_particles_mask].sum()
        filter_duration = time.time() - filter_start
    
        # Check for 10% or more of the mass in heavy particles
        if heavy_particle_mass / total_mass >= 0.05:
            logging.info(f'Halo {halo_id} skipped: 5% or more mass in heavy particles.')
            continue
    
        # Centering and orientation
        center_start = time.time()
        pynbody.analysis.halo.center(halo, mode='hyb')
        pynbody.analysis.angmom.faceon(halo)
        center_duration = time.time() - center_start
    
        # Virial radius calculation
        virial_start = time.time()
        r = pynbody.analysis.halo.virial_radius(halo, cen=None, overden=178, r_max=None, rho_def='matter')
        virial_duration = time.time() - virial_start
    
        good_halos.append(halo_id)
    
        # Log timing information
        #logging.info(f"Timing for halo {halo_id}: Load: {load_duration:.2f}s, Mass Calc: {mass_calc_duration:.2f}s, Filtering: {filter_duration:.2f}s, Centering: {center_duration:.2f}s, Virial Radius: {virial_duration:.2f}s")
    
        iteration_duration = time.time() - start_time
        logging.info(f"Total time for halo {halo_id}: {iteration_duration:.2f}s")

    except ValueError as e:
        if "Insufficient particles around center to get velocity" in str(e):
            logging.error(f'Halo {halo_id} skipped: insufficient particles for velocity calculation.')
        else:
            logging.error(f'Error processing halo {halo_id}: {e}')
    except Exception as e:
        logging.error(f'Error processing halo {halo_id}: {e}')


good_halos = np.array(good_halos,dtype=np.uintc)

np.savetxt('Storm_SIDM_good_halos.txt',good_halos,fmt='%d',delimiter = ',')


Processing Halos: 100%|█████████████████████| 8880/8880 [27:56<00:00,  5.30it/s]


In [ ]:

good_halos = np.array(good_halos,dtype=np.uintc)

np.savetxt('Storm_SIDM_good_halos.txt',good_halos,fmt='%d',delimiter = ',')